## Setup

*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

Install requirements (*Note: ocdskingfishercolab installs google-colab, which expects specific versions of pandas and numpy*):


In [ ]:
! pip install --upgrade pip > pip.log
! pip install --upgrade 'ocdskingfishercolab>=0.5,<0.6' ipywidgets psycopg2-binary >> pip.log

In [ ]:
# @title Import packages and load extensions { display-mode: "form" }

import gzip
import json
import os
import shutil
import tempfile
from collections import Counter
from datetime import datetime, timezone
from pathlib import Path

import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from google.colab.data_table import DataTable
from google.colab.files import download
from ipywidgets import widgets
from ocdskingfishercolab import (
    authenticate_gspread,
    calculate_coverage,
    download_dataframe_as_csv,
    format_thousands,
    render_json,
    save_dataframe_to_sheet,
    save_dataframe_to_spreadsheet,
    set_dark_mode,
    set_light_mode,
)

# Load https://pypi.org/project/ipython-sql/
%load_ext sql
# Load https://colab.research.google.com/notebooks/data_table.ipynb
%load_ext google.colab.data_table

In [ ]:
# @title Configure the notebook environment { display-mode: "form" }

# Increase max columns so that Pandas DataFrames with many columns are rendered as data tables.
DataTable.max_columns = 50
# Remove the index from data tables for easier copy-pasting to Google Docs.
DataTable.include_index = False

# Return Pandas DataFrames instead of regular result sets.
%config SqlMagic.autopandas = True
# Don't print number of rows affected.
%config SqlMagic.feedback = False

# If you set Tools > Settings > Site > Theme to dark, uncomment this line.
# set_dark_mode()
# If you are creating plots to copy-paste into reports, uncomment this line.
# set_light_mode()

## Charts Setup
*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

In [ ]:
! pip install altair pyarrow==14.0.0 >> pip.log

Import chart packages and define chart functions. The currently available chart functions are:

* Release count
* Objects per stage
* Releases by month
* Objects per year
* Top buyers
* Usability indicators

In [ ]:
# @title Chart functions { display-mode: "form" }
import altair as alt


class MissingColumnsError(Exception):
    def __init__(self, columns):
        super().__init__(f"The source data is missing one or more of these columns: {columns}")


chart_properties = {
    "width": 600,
    "height": 350,
    "padding": 50,
    "title": alt.TitleParams(text="", subtitle=[""], fontSize=18),
}
chart_axis = {
    "titleFontSize": 14,
    "labelFontSize": 14,
    "labelPadding": 5,
    "ticks": False,
    "domain": False,
}


def check_columns(columns, data):
    # check if input contains the right columns
    if not columns.issubset(data.columns):
        raise MissingColumnsError(columns)


def plot_release_count(release_counts):
    check_columns({"collection_id", "release_type", "release_count", "ocid_count"}, release_counts)
    return (
        alt.Chart(release_counts)
        .mark_bar()
        .encode(
            x=alt.X(
                "release_count",
                type="ordinal",
                axis=alt.Axis(title="release count", labelAngle=0),
            ),
            y=alt.Y(
                "ocid_count",
                type="quantitative",
                axis=alt.Axis(title="ocid count", format="~s", tickCount=5),
            ),
            color=alt.Color(
                "release_type",
                type="nominal",
                title="release type",
                scale=alt.Scale(range=["#D6E100", "#FB6045", "#23B2A7", "#6C75E1"]),
            ),
            tooltip=[
                alt.Tooltip("release_count", title="release count"),
                alt.Tooltip("ocid_count", title="ocid count", format="~s"),
                alt.Tooltip("release_type", title="release type"),
                alt.Tooltip("collection_id", title="collection id"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )


def plot_objects_per_stage(objects_per_stage):
    check_columns({"stage", "object_count"}, objects_per_stage)
    stages = ["planning", "tender", "awards", "contracts", "implementation"]
    return (
        alt.Chart(objects_per_stage)
        .mark_bar(fill="#d6e100")
        .encode(
            x=alt.X(
                "stage",
                type="ordinal",
                scale=alt.Scale(domain=stages),
                sort=stages,
                axis=alt.Axis(title="stage", labelAngle=0),
            ),
            y=alt.Y(
                "object_count",
                type="quantitative",
                axis=alt.Axis(title="number of objects", format="~s", tickCount=len(stages)),
            ),
            tooltip=[
                alt.Tooltip("stage", title="stage"),
                alt.Tooltip("object_count", title="number of objects"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )


def plot_releases_by_month(release_dates):
    check_columns({"date", "collection_id", "release_type", "release_count"}, release_dates)
    max_rows = 5000
    # check if number of rows is more than 5000
    if release_dates.shape[0] > max_rows:
        alt.data_transformers.disable_max_rows()

    # draw chart
    return (
        alt.Chart(release_dates)
        .mark_line(strokeWidth=3)
        .encode(
            x=alt.X("date", timeUnit="yearmonth", axis=alt.Axis(title="year and month")),
            y=alt.Y(
                "release_count",
                type="quantitative",
                aggregate="sum",
                axis=alt.Axis(title="number of releases", format="~s", tickCount=5),
                scale=alt.Scale(zero=False),
            ),
            color=alt.Color(
                "release_type",
                type="nominal",
                scale=alt.Scale(range=["#D6E100", "#FB6045", "#23B2A7", "#6C75E1"]),
                legend=alt.Legend(title="release type"),
            ),
            tooltip=[
                alt.Tooltip("date", timeUnit="yearmonth", title="date"),
                alt.Tooltip("release_count", aggregate="sum", title="number of releases"),
                alt.Tooltip("release_type", title="release type"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )


def plot_objects_per_year(objects_per_year):
    check_columns({"year", "tenders", "awards"}, objects_per_year)
    stages = ["tenders", "awards"]
    return (
        alt.Chart(objects_per_year)
        .transform_fold(stages)
        .mark_line(strokeWidth=3)
        .encode(
            x=alt.X(
                "year",
                type="quantitative",
                axis=alt.Axis(title="year", format=".0f", tickCount=objects_per_year.shape[0]),
            ),
            y=alt.Y(
                "value",
                type="quantitative",
                axis=alt.Axis(title="number of objects", format="~s", tickCount=5),
                scale=alt.Scale(zero=False),
            ),
            color=alt.Color(
                "key",
                type="nominal",
                title="object type",
                scale=alt.Scale(domain=stages, range=["#D6E100", "#FB6045"]),
            ),
            tooltip=[
                alt.Tooltip("year", title="year", type="quantitative"),
                alt.Tooltip("value", title="number of objects", type="quantitative"),
                alt.Tooltip("key", title="object type", type="nominal"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )


def plot_top_buyers(buyers):
    check_columns({"name", "total_tenders"}, buyers)
    return (
        alt.Chart(buyers)
        .mark_bar(fill="#d6e100")
        .encode(
            x=alt.X(
                "total_tenders",
                type="quantitative",
                axis=alt.Axis(title="number of tenders", format="~s", tickCount=5),
            ),
            y=alt.Y(
                "name",
                type="ordinal",
                axis=alt.Axis(title="buyer", labelAngle=0),
                sort=alt.SortField("total_tenders", order="descending"),
            ),
            tooltip=[
                alt.Tooltip("name", title="buyer", type="nominal"),
                alt.Tooltip("total_tenders", title="number of tenders", type="quantitative"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )


def plot_usability_indicators(data, lang="English"):
    labels = {
        "English": {
            "nrow": "row_number(indicator)",
            "sort": "calculation",
            "y_sort": "indicator",
            "groupby": "Use case",
            "title": "number of indicators",
            "tooltip_missing": "Missing Fields",
        },
        "Spanish": {
            "nrow": "row_number(Indicador)",
            "sort": "¿Se puede calcular?",
            "y_sort": "Indicador",
            "groupby": "Caso de Uso",
            "title": "Número de indicadores",
            "tooltip_missing": "Campos faltantes",
        },
    }
    return (
        alt.Chart(data)
        .transform_window(
            nrow=labels[lang]["nrow"],
            frame=[None, None],
            sort=[{"field": labels[lang]["sort"]}],
            groupby=[labels[lang]["groupby"]],
        )
        .mark_circle(size=250, opacity=1)
        .encode(
            x=alt.X(
                "nrow",
                type="quantitative",
                axis=alt.Axis(title=[labels[lang]["title"], ""], orient="top", tickCount=5),
            ),
            y=alt.Y(
                labels[lang]["groupby"],
                type="nominal",
                sort=alt.Sort(field=labels[lang]["y_sort"], op="count", order="descending"),
            ),
            color=alt.Color(
                labels[lang]["sort"],
                type="nominal",
                scale=alt.Scale(range=["#fb6045", "#d6e100"]),
                legend=alt.Legend(title=[labels[lang]["sort"]]),
            ),
            tooltip=[
                alt.Tooltip(labels[lang]["y_sort"], type="nominal"),
                alt.Tooltip(labels[lang]["groupby"], type="nominal"),
                alt.Tooltip(labels[lang]["sort"], type="nominal"),
                alt.Tooltip(labels[lang]["tooltip_missing"], type="nominal"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )

## Usability analysis setup

Use this section to setup the functions needed to perform a usability analysis of the dataset, to identify if a publisher has the necessary fields to calculate 71 procurement indicators related to market opportunity (market description, competition, supplier performance), value for money, internal efficiency, public integrity and service delivery.  For an OCDS publisher, it also calculates the proportion of unique procedures for which it is possible to calculate the indicator (coverage).

The usability checks includes all the indicators listed on [OCP's use case guide](https://docs.google.com/spreadsheets/d/1j-Y0ktZiOyhZzi-2GSabBCnzx6fF5lv8h1KYwi_Q9GM/edit#gid=1183427361) and the [Indicators to diagnose the performance of a procurement market document](https://docs.google.com/document/d/1vSJk9-qWSTQEx9ZZc7BUhQZMHvTRcyDYVS2sl8HB__k/edit#heading=h.nrnq1ajwwpqe).

In [ ]:
# @title Usability functions { display-mode: "form" }

RELEVANT_RULES = {
    "who": [
        "buyer/id",
        "buyer/name",
        "tender/procuringEntity/id",
        "tender/procuringEntity/name",
    ],
    "bought what": [
        "tender/items/classification/id",
        "awards/items/classification/id",
        "contracts/items/classification/id",
        "tender/items/classification/description",
        "awards/items/classification/description",
        "contracts/items/classification/description",
        "tender/items/description",
        "awards/items/description",
        "contracts/items/description",
        "tender/description",
        "awards/description",
        "contracts/description",
        "tender/title",
        "awards/title",
        "contracts/title",
    ],
    "from whom": [
        "awards/suppliers/id",
        "awards/suppliers/name",
    ],
    "for how much": [
        "awards/value/amount",
        "contracts/value/amount",
        [
            "awards/items/quantity",
            "awards/items/unit/value/amount",
        ],
        [
            "contracts/items/quantity",
            "contracts/items/unit/value/amount",
        ],
    ],
    "when": [
        "tender/tenderPeriod/endDate",
        "awards/date",
        "contracts/dateSigned",
    ],
    "how": [
        "tender/procurementMethod",
        "tender/procurementMethodDetails",
    ],
}


def get_indicators_dictionary(fields_list):
    """
    Check which alternative fields are available for indicators.

    For example, the number of tenderers can use either `tender/numberOfTenderers` or `tender/tenderers/id`.
    """
    # U002
    buyer = ["buyer/name", "buyer/id"]
    procuring = ["tender/procuringEntity/name", "tender/procuringEntity/id"]
    parties = ["parties/identifier/name", "parties/identifier/id", "parties/roles"]
    if not any(item not in fields_list for item in buyer):
        buyer_var = buyer
    elif not any(item not in fields_list for item in procuring):
        buyer_var = procuring
    elif not any(item not in fields_list for item in parties):
        buyer_var = parties
    else:
        buyer_var = buyer

    # U003
    if "tender/tenderers/id" in fields_list:
        bidders_val = "tender/tenderers/id"
    elif "bids/details/tenderers/id" in fields_list:
        bidders_val = "bids/details/tenderers/id"
    else:
        bidders_val = "tender/tenderers/id"

    # U008
    if "tender/items/classification/id" in fields_list and "tender/items/classification/scheme" in fields_list:
        items_val = ["tender/items/classification/id", "tender/items/classification/scheme"]
    elif "awards/items/classification/id" in fields_list and "awards/items/classification/scheme" in fields_list:
        items_val = ["awards/items/classification/id", "awards/items/classification/scheme"]
    elif "contracts/items/classification/id" in fields_list and "contractsitems/classification/scheme" in fields_list:
        items_val = ["contracts/items/classification/id", "contracts/items/classification/scheme"]
    else:
        items_val = ["tender/items/classification/id", "tender/items/classification/scheme"]

    # U012
    if "contracts/id" in fields_list and "contracts/status" in fields_list:
        awards_val = ["contracts/id", "contracts/status"]
    elif "awards/id" in fields_list and "awards/status" in fields_list:
        awards_val = ["awards/id", "awards/status"]
    else:
        awards_val = ["contracts/id", "contracts/status"]

    # U013, UC14
    awards = ["awards/id", "awards/status", "awards/value/amount", "awards/value/currency"]
    contracts = ["contracts/id", "contracts/status", "contracts/value/amount", "contracts/value/currency"]
    if not any(item not in fields_list for item in contracts):
        awards_val2 = contracts
    elif not any(item not in fields_list for item in awards):
        awards_val2 = awards
    else:
        awards_val2 = contracts

    # U015
    if "tender/numberOfTenderers" in fields_list:
        bidders_val2 = "tender/numberOfTenderers"
    elif "tender/tenderers/id" in fields_list:
        bidders_val2 = "tender/tenderers/id"
    elif "bids/details/tenderers/id" in fields_list:
        bidders_val2 = "bids/details/tenderers/id"
    else:
        bidders_val2 = "tender/numberOfTenderers"

    # U034
    aw = [
        "awards/status",
        "awards/value/amount",
        "awards/value/currency",
        "awards/items/classification/id",
        "awards/items/classification/scheme",
    ]
    con = [
        "contracts/status",
        "contracts/value/amount",
        "contracts/value/currency",
        "contracts/items/classification/id",
        "contracts/items/classification/scheme",
    ]
    if not any(item not in fields_list for item in aw):
        awards_val3 = aw
    elif not any(item not in fields_list for item in con):
        awards_val3 = con
    else:
        awards_val3 = aw

    # U042
    if "awards/status" in fields_list:
        awards_val4 = "awards/status"
    elif "contracts/status" in fields_list:
        awards_val4 = "contracts/status"
    else:
        awards_val4 = "awards/status"

    # U061
    if "contracts/period/startDate" in fields_list:
        contract_date = "contracts/period/startDate"
    elif "awards/contractPeriod/startDate" in fields_list:
        contract_date = "awards/contractPeriod/startDate"
    else:
        contract_date = "contracts/period/startDate"

    # U065
    aw2 = ["awards/items/quantity", "awards/items/unit"]
    con2 = ["contracts/items/quantity", "contracts/items/unit"]
    if not any(item not in fields_list for item in aw):
        awards_val5 = aw2
    elif not any(item not in fields_list for item in con):
        awards_val5 = con2
    else:
        awards_val5 = aw2

    # U066, U067
    if "planning/budget/amount/amount" in fields_list and "planning/budget/amount/currency":
        planning = ["planning/budget/amount/amount", "planning/budget/amount/currency"]
    elif "tender/value/amount" in fields_list and "tender/value/currency":
        planning = ["tender/value/amount", "tender/value/currency"]
    else:
        planning = ["planning/budget/amount/amount", "planning/budget/amount/currency"]

    return {
        "Total number of procedures": [["U001"], ["ocid"]],
        "Total number of procuring entities": [["U002"], ["ocid", *buyer_var]],
        "Total number of unique bidders": [["U003"], ["ocid", bidders_val]],
        "Total number of awarded suppliers": [
            ["U004"],
            ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/status"],
        ],
        "Total number of procedures by year or month": [["U005"], ["ocid", "date"]],
        "Total value awarded": [["U006"], ["ocid", "awards/status", "awards/value/amount", "awards/value/currency"]],
        "Share of procedures by status": [["U007"], ["ocid", "tender/status"]],
        "Number of procedures by item type": [["U008"], ["ocid", *items_val]],
        "Proportion of procedures by procurement category": [["U009"], ["ocid", "tender/mainProcurementCategory"]],
        "Percent of tenders by procedure type": [["U010"], ["ocid", "tender/procurementMethod"]],
        "Percent of tenders awarded by means of competitive procedures": [
            ["U011"],
            ["ocid", "tender/procurementMethod", "awards/status"],
        ],
        "Percent of contracts awarded under each procedure type": [
            ["U012"],
            ["ocid", "tender/procurementMethod", *awards_val],
        ],
        "Total contracted value awarded under each procedure type": [
            ["U013"],
            ["ocid", "tender/procurementMethod", *awards_val2],
        ],
        "Total awarded value of tenders awarded by means of competitive procedures": [
            ["U014"],
            ["ocid", "tender/procurementMethod", *awards_val2],
        ],
        "Proportion of single bid tenders": [["U015"], ["ocid", "tender/procurementMethod", bidders_val2]],
        "Proportion of value awarded in single bid tenders vs competitive tenders": [
            ["U016"],
            [
                "ocid",
                "tender/procurementMethod",
                "awards/status",
                "awards/value/amount",
                "awards/value/currency",
                bidders_val2,
            ],
        ],
        "Mean number of bidders per tender": [["U017"], ["ocid", "tender/procurementMethod", bidders_val2]],
        "Median number of bidders per tender": [["U018"], ["ocid", "tender/procurementMethod", bidders_val2]],
        "Mean number of bidders by item type": [
            ["U019"],
            ["ocid", "tender/procurementMethod", bidders_val2, *items_val],
        ],
        "Number of suppliers by item type": [
            ["U020"],
            ["awards/id", "awards/suppliers/id", "awards/suppliers/name", *items_val],
        ],
        "Number of new bidders in a system ": [
            ["U021"],
            ["tender/id", "tender/tenderers/id", "tender/tenderPeriod/startDate"],
        ],
        "Percent of new bidders to all bidders ": [
            ["U022"],
            ["tender/id", "tender/tenderers/id", "tender/tenderPeriod/startDate"],
        ],
        "Percent of tenders with at least three participants deemed qualified": [
            ["U023"],
            ["ocid", "bids/details/tenderers/id", "bids/details/id", "bids/details/status"],
        ],
        "Mean percent of bids which are disqualified": [
            ["U024"],
            ["tender/id", "bids/details/id", "bids/details/status"],
        ],
        "Percent of contracts awarded to top 10 suppliers with largest contracted totals": [
            ["U025"],
            ["awards/id", "awards/suppliers/id", "awards/suppliers/name", *awards_val2],
        ],
        "Mean number of unique suppliers per buyer": [
            ["U026"],
            ["ocid", "awards/suppliers/id", "awards/suppliers/name", *buyer_var],
        ],
        "Number of new awarded suppliers ": [
            ["U027"],
            ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"],
        ],
        "Percent of awards awarded to new suppliers": [
            ["U028"],
            ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"],
        ],
        "Total awarded value awarded to new suppliers": [
            ["U029"],
            [
                "awards/id",
                "awards/suppliers/id",
                "awards/suppliers/name",
                "awards/date",
                "awards/value/amount",
                "awards/value/currency",
            ],
        ],
        "Percent of new suppliers to all suppliers": [
            ["U030"],
            ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"],
        ],
        "Percent of growth of new awarded suppliers in a system": [
            ["U031"],
            ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"],
        ],
        "Percent of total awarded value awarded to recurring suppliers": [
            ["U032"],
            [
                "awards/id",
                "awards/suppliers/id",
                "awards/suppliers/name",
                "awards/date",
                "awards/value/amount",
                "awards/value/currency",
            ],
        ],
        "Mean number of bids necessary to win": [
            ["U033"],
            ["ocid", "tender/tenderers/id", "awards/suppliers/id", "awards/suppliers/name"],
        ],
        "Market concentration, market share of the largest company in the market": [
            ["U034"],
            ["awards/suppliers/id", "awards/suppliers/name", *awards_val3],
        ],
        "Proportion of contracts awarded by supplier by non competitive procedures": [
            ["U035"],
            ["ocid", "tender/procurementMethod", "awards/status", "awards/suppliers/id", "awards/suppliers/name"],
        ],
        "Region of the supplier": [["U036"], ["parties/roles", "parties/identifier/id", "parties/address/region"]],
        "Number of bids submitted by supplier": [["U037"], ["awards/suppliers/id", bidders_val]],
        "Success rate of bidders": [
            ["U038"],
            ["ocid", "tender/tenderers/id", "awards/suppliers/id", "awards/suppliers/name"],
        ],
        "Number of unique items classifications awarded by supplier": [
            ["U039"],
            ["awards/id", "awards/suppliers/id", "awards/suppliers/name", *items_val],
        ],
        "Total value awarded by supplier": [["U040"], ["awards/suppliers/id", "awards/suppliers/name", *awards_val2]],
        "Share of total value awarded by supplier": [
            ["U041"],
            ["awards/suppliers/id", "awards/suppliers/name", *awards_val2],
        ],
        "Total number of contracts awarded by supplier": [
            ["U042"],
            ["awards/id", "awards/suppliers/id", "awards/suppliers/name", awards_val4],
        ],
        "Number of procuring entities by supplier": [
            ["U043"],
            ["ocid", "awards/suppliers/id", "awards/suppliers/name", *buyer_var],
        ],
        "Share of single bid awards by supplier": [
            ["U044"],
            [
                "ocid",
                "awards/suppliers/id",
                "awards/suppliers/name",
                "awards/status",
                "tender/procurementMethod",
                bidders_val2,
            ],
        ],
        "Percent of tenders with linked procurement plans": [["U045"], ["tender/id", "tender/documents/documentType"]],
        "Percent of contracts which publish information on debarments": [
            ["U046"],
            ["contracts/id", "contracts/implementation/documents/documentType"],
        ],
        "The percent of tenders for which the tender documentation was added after publication of the announcement ": [
            ["U047"],
            [
                "tender/id",
                "tender/documents/documentType",
                "tender/documents/documentType",
                "tender/documents/datePublished",
            ],
        ],
        "Mean number of contract amendments per buyer": [
            ["U048"],
            ["ocid", "contracts/id", "contracts/amendments", *buyer_var],
        ],
        "Percent of tenders which have been closed for more than 30 days, but whose basic awards information is not published": [
            ["U049"],
            [
                "tender/id",
                "tender/tenderPeriod/endDate",
                "awards/id",
                "awards/date",
                "awards/status",
                "awards/value/amount",
                "awards/suppliers/id",
                "awards/suppliers/name",
            ],
        ],
        "Percent of awards which are older than 30 days, but whose contract is not published": [
            ["U050"],
            [
                "awards/id",
                "awards/date",
                "contracts/awardID",
                "contracts/status",
                "contracts/dateSigned",
                "contracts/documents/documentType",
            ],
        ],
        "Percent of tenders that do not specify place of delivery": [
            ["U051"],
            ["ocid", "tender/items/deliveryLocation", "tender/items/deliveryAddress"],
        ],
        "Percent of tenders that do not specify date of delivery": [
            ["U052"],
            [
                "tender/milestones/id",
                "tender/milestones/type",
                "tender/milestones/description",
                "tender/milestones/dueDate",
            ],
        ],
        "Percent of tenders with short titles for example fewer than 10 characters in the title": [
            ["U053"],
            ["tender/id", "tender/title"],
        ],
        "Percent of tenders with short descriptions for instance fewer than 30 characters in the description": [
            ["U054"],
            ["tender/id", "tender/description"],
        ],
        "Percent of tenders that do not include detailed item codes or item descriptions": [
            ["U055"],
            ["tender/id", "tender/items/classification/id", "tender/items/classification/scheme"],
        ],
        "Percent of contracts that do not have amendments": [["U056"], ["contracts/id", "contracts/amendments"]],
        "Percent of contracts which publish contract implementation details financial": [
            ["U057"],
            [
                "contracts/implementation/transactions/id",
                "contracts/implementation/transactions/value/amount",
                "contracts/implementation/transactions/value/currency",
            ],
        ],
        "Percent of contracts which publish contract implementation details physical": [
            ["U058"],
            [
                "contracts/implementation/milestones/type",
                "contracts/implementation/milestones/id",
                "contracts/implementation/milestones/dueDate",
                "contracts/implementation/milestones/status",
            ],
        ],
        "Average duration of tendering period days": [
            ["U059"],
            ["ocid", "tender/tenderPeriod/startDate", "tender/tenderPeriod/endDate"],
        ],
        "Average duration of decision period days": [["U060"], ["ocid", "tender/tenderPeriod/endDate", "awards/date"]],
        "Average days from award date to start of implementation": [
            ["U061"],
            ["awards/id", "awards/date", contract_date],
        ],
        "Days between award date and tender start date": [
            ["U062"],
            ["ocid", "tender/tenderPeriod/startDate", "awards/date"],
        ],
        "Percent of canceled tenders to awarded tenders": [["U063"], ["ocid", "tender/status", "awards/status"]],
        "Percent of contracts which are canceled": [["U064"], ["contracts/id", "contracts/status"]],
        "Price variation of same item across all awards": [["U065"], awards_val3 + awards_val5],
        "Percent of contracts that exceed budget": [
            ["U066"],
            ["ocid", "contracts/status", "contracts/value/amount", "contracts/value/currency", *planning],
        ],
        "Mean percent overrun of contracts that exceed budget": [
            ["U067"],
            ["ocid", "contracts/status", "contracts/value/amount", "contracts/value/currency", *planning],
        ],
        "Total percent savings difference between budget and contract value": [
            ["U068"],
            [
                "ocid",
                "planning/budget/amount/amount",
                "planning/budget/amount/currency",
                "contracts/value/amount",
                "contracts/value/currency",
            ],
        ],
        "Total percent savings difference between tender value estimate and contract value": [
            ["U069"],
            [
                "ocid",
                "tender/value/amount",
                "tender/value/currency",
                "contracts/value/amount",
                "contracts/value/currency",
            ],
        ],
        "Percent of contracts completed on time ": [
            ["U070"],
            ["contracts/id", "contracts/period/endDate", "contracts/status"],
        ],
        "Share of contracts whose milestones are completed on time": [
            ["U071"],
            [
                "contracts/id",
                "contracts/implementation/milestones/dueDate",
                "contracts/implementation/milestones/dateMet",
            ],
        ],
    }


def usability_checks(fields_list, indicators_dic):
    """
    Return a table of the usability checks.

    It indicates if the fields needed to calculate a particular indicator are present.
    Set check_coverage=True to check for coverage.
    """
    results_list = []
    missing_fields = []

    for i in indicators_dic.values():
        check = any(item not in fields_list for item in i[1])
        result = "missing fields" if check else "possible to calculate"
        missing = [i[1][j] for j in range(len(i[1])) if i[1][j] not in fields_list]
        missing_fields.append(missing)
        results_list.append(result)

    # Generate dataframe

    indicatordf = pd.DataFrame(
        list(
            zip(
                list(indicators_dic),
                [indicators_dic[i][0] for i in indicators_dic],
                [indicators_dic[i][1:] for i in indicators_dic],
                strict=True,
            )
        ),
        columns=["indicator", "U_id", "fields needed"],
    )
    indicatordf["U_id"] = indicatordf["U_id"].apply(lambda x: ", ".join(map(str, x)))
    indicatordf["fields needed"] = indicatordf["fields needed"].astype(str).str.replace(r"\[|\]|'", "", regex=True)
    indicatordf["calculation"] = results_list
    indicatordf["missing fields"] = missing_fields
    indicatordf["missing fields"] = indicatordf["missing fields"].apply(lambda x: ", ".join(map(str, x)))

    return indicatordf


def check_usability_indicators(lang, result):

    gc = authenticate_gspread()

    if lang.value == "English":
        # Use case guide: Indicators linked to OCDS #public
        spreadsheet_key = "1j-Y0ktZiOyhZzi-2GSabBCnzx6fF5lv8h1KYwi_Q9GM"
    else:
        # [ES] of Use case guide: Indicators linked to OCDS #public
        spreadsheet_key = "1l_p_e1iNUUuR5AObTJ8EY9VrcCLTAq3dnG_Fj73UH9w"

    worksheet = gc.open_by_key(spreadsheet_key).sheet1

    # get_all_values gives a list of rows.
    rows = worksheet.get_all_values()
    # Convert to a DataFrame and render.

    indicators = pd.DataFrame(rows)
    indicators = indicators.rename(columns=indicators.iloc[0]).drop(indicators.index[0])

    if lang.value == "English":
        indicatorsdf = indicators.iloc[:, [0, 3, 4, 9]]
        result_final = result.merge(indicatorsdf, on="U_id")
    else:
        indicatorsdf = indicators.iloc[:, [0, 3, 4, 5, 9]]
        result_final = indicatorsdf.merge(result, on="U_id").drop(["indicator"], axis=1)
        result_final = result_final.rename(
            columns={
                "fields needed": "Campos necesarios",
                "calculation": "¿Se puede calcular?",
                "missing fields": "Campos faltantes",
                "coverage": "Cobertura",
            },
        )
        result_final = result_final.replace(
            {"¿Se puede calcular?": {"possible to calculate": "sí", "missing fields": "campos faltantes"}}
        )
    return result_final


def is_relevant(field_list):
    final_result = []
    for key, value in RELEVANT_RULES.items():
        rule_result = {"rule": key, "possible_to_calculate": "No", "available_fields": [], "missing_fields": []}
        for field in value:
            if isinstance(field, str):
                if field in field_list:
                    rule_result["possible_to_calculate"] = "Yes"
                    rule_result["available_fields"].append(field)
                else:
                    rule_result["missing_fields"].append(field)
            else:
                missing = [item for item in field if item not in field_list]
                rule_result["available_fields"].extend(item for item in field if item in field_list)
                rule_result["missing_fields"].extend(missing)
                if not missing:
                    rule_result["possible_to_calculate"] = "Yes"
        final_result.append(rule_result)

    final_result = pd.DataFrame(final_result)
    relevant = (final_result["possible_to_calculate"] == "Yes").all()
    return relevant, final_result


def get_coverage(indicators_dic):
    coverage = []
    for i in indicators_dic.values():
        fields = [item for sublist in i for item in sublist][1:]
        result = calculate_coverage(fields, "release_summary")
        result_value = pd.to_numeric(result["total_percentage"][0])
        coverage.append(result_value)
    return coverage


def most_common_fields_to_calculate_indicators(indicators_dict, fields_table):
    fields = list(indicators_dict.values())
    fields = [item[1:] for item in fields]
    flat_list = [item for sublist in [item for sublist in fields for item in sublist] for item in sublist]
    fields_list = Counter(flat_list)

    fields_count = (
        pd.DataFrame.from_dict(fields_list, orient="index")
        .reset_index()
        .rename(columns={"index": "field", 0: "number of indicators"})
    )

    fields_count = fields_count.sort_values("number of indicators", ascending=False).reset_index(drop=True)
    fields_count["published"] = np.where(fields_count["field"].isin(fields_table["path"]), "yes", "no")

    return fields_count


def get_usability_language_select_box():
    style = {"description_width": "initial"}
    languages = ["Spanish", "English"]
    return widgets.Dropdown(options=languages, description="language", style=style)

## Setup Kingfisher Process

### Connect to the database

In [ ]:
import getpass

from ocdskingfishercolab import (
    list_collections,
    list_source_ids,
    set_search_path,
)

Enter your PostgreSQL credentials and connect to the Kingfisher Process database:

In [ ]:
user = input("Username:")
password = getpass.getpass("Password:")

# Don't show connection string after execute.
%config SqlMagic.displaycon = False

connection_string = (
    "postgresql://"
    + user
    + ":"
    + password
    + "@postgres.kingfisher.open-contracting.org/kingfisher_process?sslmode=require"
)
%sql $connection_string

### Choose collections and schema

*Use this section to choose the collections and schema that you want to query.*

#### Set the collection(s)

Update `collection_ids` with the `id`(s) of the [Kingfisher Process collection(s)](https://kingfisher-process.readthedocs.io/en/latest/data-model.html#collections):

In [ ]:
collection_ids = (2358, 2359)

If you don't know which collections you need, run the next cell and use the **Filter** button to filter the [collection table](https://kingfisher-process.readthedocs.io/en/latest/database-structure.html#collection-table) to find the collection(s). You can use the `source_id` column to filter on the `name` of the [Kingfisher Collect spider](https://kingfisher-collect.readthedocs.io/en/latest/spiders.html) used to collect the data. Use the value(s) from the `id` column to update the previous cell.

In [ ]:
list_collections()

#### Set the schema

Update `schema_name` with the name of the [Kingfisher Summarize schema](https://kingfisher-summarize.readthedocs.io/en/latest/index.html#how-it-works).

In [ ]:
schema_name = "view_data_collection_2358_2359"
set_search_path(schema_name)

If you don't know which schema you need, run the next cell and use the **Filter** button to filter the [selected collections table](https://kingfisher-summarize.readthedocs.io/en/latest/database.html#summaries-selected-collections) to find the schema. You can use the `collection_id` column to filter on the `id` of the collections that you identified in the previous step. Alternatively, you can filter on the `source_id` column. Use the value from the `schema` column to update the previous cell.

In [ ]:
%%sql
SELECT
    summaries.selected_collections.*,
    source_id
FROM
    summaries.selected_collections
INNER JOIN
    collection
    ON summaries.selected_collections.collection_id = collection.id


If you can't find a schema containing the collections that you want to query, you can create a schema using [Kingfisher Summarize](https://ocdsdeploy.readthedocs.io/en/latest/use/kingfisher-summarize.html).

## Check for data collection and processing errors

Kingfisher Collect and Kingfisher Process log messages that might indicate OCDS implementation errors or API stability issues.

Confirm any errors or warnings by manually checking the OCDS publication.

### Kingfisher Collect

See [how to review the Kingfisher Collect crawl’s log file](https://kingfisher-collect.readthedocs.io/en/latest/logs.html).

**Note:** There is an open [pull request](https://github.com/open-contracting/notebooks-ocds/pull/44) to automate this. If interested, please comment on the pull request to prioritize it.

### Kingfisher Process

Users add a note when starting a crawl and when running the `load` command.

Kingfisher Process adds notes (the `note` column) at different levels (the `code` column):

- `INFO`
  - *load*: The Kingfisher Collect crawl's reason for closing and statistics (the `data` column).
- `WARNING`
  - *compile*: When a record has undated releases, linked releases or no releases, but a compiled release can be calculated (by merging remaining dated releases, using the `compiledRelease` field, or using an undated release with a 'compiled' tag).
- `ERROR`
  - *load*: When the input data is invalid JSON.
  - *compile*: When no compiled release can be calculated for a record.

📗 **You can check the full list of note types and what they mean in the Kingfisher Process's [documentation](https://kingfisher-process.readthedocs.io/en/latest/database.html#collection-note-table)**📗


List the notes for each of your collections:

In [ ]:
%%sql
SELECT
    collection_id,
    code,
    note,
    data
FROM
    collection_note
WHERE
    collection_id IN :collection_ids


## Check for structure and format errors

Kingfisher Collect reformats data sources as release packages or record packages. Check the `data_type` class attribute of the [spider](https://github.com/open-contracting/kingfisher-collect/tree/main/kingfisher_scrapy/spiders). If it is not 'release_package' or 'record_package', recommend to the publisher to package their data.

Kingfisher Process checks data against the OCDS schema using [lib-cove-ocds](https://github.com/open-contracting/lib-cove-ocds), same as the [OCDS Data Review Tool](https://review.standard.open-contracting.org). For release collections, Kingfisher Process stores check results in the `release_check` table. For record collections, Kingfisher Process stores check results in the `record_check` table.

### Confirm that checks are complete

If a crawl is scheduled using Kingfisher Collect, by default, Kingfisher Process performs structural checks. Checking data is the slowest step. For large collections, it is recommended to skip the `check` step or to collect only a sample. Otherwise, there can be a backlog of data to check.

Use this query to determine whether checks are complete for your collection(s).

If checks are in progress, you should wait for the checks to finish before running the queries in this section.

In [ ]:
%%sql
SELECT
    collection_id,
    'release' AS collection_type,
    CASE
        WHEN count(release.id) = count(release_check.id)
            THEN
                'complete'
        WHEN count(release_check.id) = 0
            THEN
                'not_started'
        ELSE
            'in_progress'
    END AS check_status,
    count(release_check.id)::text
    || '/'
    || count(release.id)::text AS check_progress
FROM
    release_check
RIGHT JOIN release ON release_check.release_id = release.id
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id
UNION
SELECT
    collection_id,
    'record' AS collection_type,
    CASE
        WHEN count(record.id) = count(record_check.id)
            THEN
                'complete'
        WHEN count(record_check.id) = 0
            THEN
                'not_started'
        ELSE
            'in_progress'
    END AS check_status,
    count(record_check.id)::text
    || '/'
    || count(record.id)::text AS check_progress
FROM
    record_check
RIGHT JOIN record ON record_check.record_id = record.id
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id;


### Error summary

Summarize the errors from the `release_check` and `record_check` tables.

In [ ]:
%%sql structure_and_format_error_summary <<
WITH errors AS (
    SELECT
        collection_id,
        errors ->> 'type' AS error_type,
        left(
            errors ->> 'description',
            49000
        ) AS error,
        ocid,
        errors ->> 'field' AS field,
        errors ->> 'value' AS value,
        row_number() OVER (
            PARTITION BY
                collection_id,
                errors ->> 'type',
                left(
                    errors ->> 'description',
                    49000
                )
        ) AS rownum
    FROM
        release_check AS rc
    CROSS JOIN
        jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    INNER JOIN release AS r ON rc.release_id = r.id
    WHERE
        collection_id IN :collection_ids
    UNION ALL
    SELECT
        collection_id,
        errors ->> 'type' AS error_type,
        left(
            errors ->> 'description',
            49000
        ) AS error,
        ocid,
        errors ->> 'field' AS field,
        errors ->> 'value' AS value,
        row_number() OVER (
            PARTITION BY
                collection_id,
                errors ->> 'type',
                left(
                    errors ->> 'description',
                    49000
                )
        ) AS rownum
    FROM
        record_check AS rc
    CROSS JOIN
        jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    INNER JOIN record AS r ON rc.record_id = r.id
    WHERE
        collection_id IN :collection_ids
),

examples AS (
    SELECT
        collection_id,
        error_type,
        field,
        error,
        array_agg(ocid) AS example_ocids,
        array_agg(value) AS example_values
    FROM
        errors
    WHERE
        rownum <= 3
    GROUP BY
        collection_id,
        error_type,
        field,
        error
)

SELECT
    collection_id,
    error_type,
    errors.field,
    error,
    count(*) AS count,
    example_ocids,
    example_values
FROM
    errors
INNER JOIN examples USING (collection_id, error_type, error)
GROUP BY
    collection_id,
    error_type,
    errors.field,
    error,
    example_ocids,
    example_values;


In [ ]:
structure_and_format_error_summary

### Error details

List all errors from the `release_check` and `record_check` tables.

In [ ]:
%%sql structure_and_format_errors <<
SELECT
    collection_id,
    'release' AS collection_type,
    errors ->> 'type' AS error_type,
    errors ->> 'field' AS field,
    left(
        errors ->> 'description',
        49000
    ) AS error,
    ocid,
    errors ->> 'value' AS value
FROM
    release_check AS rc
CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
INNER JOIN release AS r ON rc.release_id = r.id
WHERE
    collection_id IN :collection_ids
UNION ALL
SELECT
    collection_id,
    'record' AS collection_type,
    errors ->> 'type' AS error_type,
    errors ->> 'field' AS field,
    left(
        errors ->> 'description',
        49000
    ) AS error,
    ocid,
    errors ->> 'value' AS value
FROM
    record_check AS rc
CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
INNER JOIN record AS r ON rc.record_id = r.id
WHERE
    collection_id IN :collection_ids


In [ ]:
structure_and_format_errors

## Perform manual conformance checks

[Conforming publications](https://standard.open-contracting.org/latest/en/schema/conformance_and_extensions/#publication-conformance):

* Use a [registered OCID prefix](https://standard.open-contracting.org/latest/en/schema/identifiers/#contracting-process-identifier-ocid).
* The [OCDS Data Review Tool](https://standard.open-contracting.org/review/) can report results on the data.
* Must not use terms from outside the OCDS schema where its terms would suffice.

Use this section to:

* Check if the proper OCID prefix is in use.
* Check if the OCDS Data Review Tool is able to report results on data.
* Identify fields in local extensions and additional fields and codes that should be mapped to fields and codes in the OCDS schema and extensions.
* Identify deprecated fields, in order to recommend a change to the publisher's OCDS implementation.


You can use the following resources to find fields and codes with similar semantics:

* [OCDS Schema and Codelist Reference](https://standard.open-contracting.org/latest/en/schema/), for fields in the core OCDS schema.
* [OCDS Extensions Field and Code Search](https://open-contracting.github.io/editor-tools/), for fields and codes in OCDS extensions.
* [GitHub Issue Tracker](https://github.com/open-contracting/standard/issues), for discussions about adding new fields and codes.

Check that field and code names [conform to the style guide](https://ocds-standard-development-handbook.readthedocs.io/en/latest/meta/schema_style_guide.html#field-and-code-names) and report any issues to the publisher.

If you cannot find a suitable mapping for an additional field or code, [open a GitHub issue](https://github.com/open-contracting/standard/issues) to describe the semantics of the field or code and to discuss how to model it. Report any issues to the publisher.

**Note:** This section depends on the `check` step of Kingfisher Process having completed. See *Check for structure and format errors >Confirm that checks are complete*. If the checks are not complete or the OCDS Data Review Tool is not able to report results on data, **the publisher don't pass that conformance criterion**

### OCID prefix

Check that the data uses the OCID prefix that was issued to this specific publisher.

You only need to run either the *Release prefixes* or *Record prefixes* section, depending on the publication's data format.

Update the ocid prefix in the appropriate cell. Prefixes can be found at [the list of registered prefixes](https://docs.google.com/spreadsheets/d/1E5ZVhc8VhGOakCq4GegvkyFYT974QQb-sSjvOfaxH7s/pubhtml?gid=506986894&single=true&widget=true).

#### **Release prefixes**

Notify the publisher of any incorrect prefixes.

In [ ]:
# Do not remove the final % character
ocid_prefix = "%"

In [ ]:
%%sql ocid_prefix_release_check <<
SELECT ocid
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
    AND ocid NOT LIKE :ocid_prefix


In [ ]:
ocid_prefix_release_check

#### **Record prefixes**

Notify the publisher of any incorrect prefixes.

In [ ]:
%%sql ocid_prefix_record_check <<
SELECT ocid
FROM
    record_summary
WHERE
    collection_id IN :collection_ids
    AND ocid NOT LIKE :ocid_prefix


In [ ]:
ocid_prefix_record_check

### Local extensions

For each field and code in extensions authored by the publisher, in addition to the above checks, consider whether to [review the extension in detail](https://docs.google.com/document/d/1CS_TMubqoYaucT8JXPTgLS-mF4eMIifX-6mD0xpWg9M/edit).

List the extensions declared in the package metadata.

**Note:** This query should be kept in sync with the query in *Check scope > Extensions*.


In [ ]:
%%sql
SELECT
    collection_id,
    release_type,
    jsonb_array_elements(package_data -> 'extensions') AS ocds_extension,
    count(*) AS count
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
    AND package_data IS NOT NULL
GROUP BY
    collection_id,
    release_type,
    ocds_extension
ORDER BY
    collection_id ASC,
    release_type ASC,
    count DESC;


### Additional fields

[lib-cove-ocds](https://github.com/open-contracting/lib-cove-ocds) reports additional fields in the following scenarios:

* Fields from undeclared extensions.
* Fields with language variations, e.g. `title_es`. You do not need to report language variations to the publisher, but you should check that the field [conforms to the rules for language variations](https://standard.open-contracting.org/latest/en/schema/reference/#language).
* OCDS 1.0 data using extension fields. You should report the fields to the publisher and recommend that they upgrade to OCDS 1.1.


List additional fields.

By default, results are reported for a sample of 10% of releases. For small collections, you can set `sample_size` to `1` to return results for the full collection. For large collections, you can reduce the sample size.

In [ ]:
sample_size = 0.1

In [ ]:
%%sql
WITH check_results AS (
    SELECT
        *,
        CASE
            WHEN (release_type IN ('record', 'embedded_release'))
                THEN
                    record_check
            ELSE
                release_check
        END AS results
    FROM
        release_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type <> 'compiled_release'
        AND random() < :sample_size
),

counts AS (
    SELECT
        collection_id,
        release_type,
        additional_fields ->> 'path' AS path,
        additional_fields ->> 'field_name' AS field,
        sum((additional_fields ->> 'count')::int) AS count
    FROM
        check_results
    CROSS JOIN
        jsonb_array_elements(
            results -> 'all_additional_fields'
        ) AS additional_fields
    GROUP BY
        collection_id,
        release_type,
        field,
        path
    ORDER BY
        path ASC,
        count DESC
),

examples AS (
    SELECT DISTINCT ON (
        collection_id,
        release_type,
        additional_fields ->> 'path',
        additional_fields ->> 'field_name')
        collection_id,
        release_type,
        additional_fields ->> 'path' AS path,
        additional_fields ->> 'field_name' AS field,
        additional_fields ->> 'examples' AS examples
    FROM
        check_results
    CROSS JOIN
        jsonb_array_elements(
            results -> 'all_additional_fields'
        ) AS additional_fields
    WHERE
        jsonb_array_length(additional_fields -> 'examples') > 0
)

SELECT
    counts.collection_id,
    counts.release_type,
    counts.path,
    counts.field,
    count,
    examples
FROM
    counts
LEFT JOIN examples
    USING (
        collection_id,
        release_type,
        path,
        field
    )
ORDER BY
    path,
    field;


#### Additional field examples

Generate a release package containing an example release for each additional field:

In [ ]:
%%sql additional_field_examples <<
WITH additional_field_releases AS (
    SELECT
        ocid,
        release.release_id,
        data_id,
        additional_fields ->> 'path' AS path,
        additional_fields ->> 'field_name' AS field
    FROM
        release_check
    CROSS JOIN
        jsonb_array_elements(
            cove_output -> 'all_additional_fields'
        ) AS additional_fields
    INNER JOIN release ON release_check.release_id = release.id
    WHERE
        collection_id IN :collection_ids
        AND random() < :sample_size
),

additional_fields AS (
    SELECT DISTINCT
        path,
        field
    FROM
        additional_field_releases
),

examples AS (
    SELECT DISTINCT ON (
        additional_fields.path,
        additional_fields.field)
        additional_fields.path,
        additional_fields.field,
        ocid,
        release_id,
        data_id,
        data
    FROM
        additional_fields
    INNER JOIN additional_field_releases
        ON
            additional_fields.path = additional_field_releases.path
            AND additional_fields.field = additional_field_releases.field
    INNER JOIN data ON data.id = data_id
    ORDER BY
        additional_fields.path,
        additional_fields.field
)

SELECT jsonb_build_object('releases', jsonb_agg(data)) AS release_package
FROM
    examples


In [ ]:
render_json(additional_field_examples["release_package"][0])

### Additional codes

List additional codes in the context of an open codelist.

Using additional codes in the context of a closed codelist is an error, and is reported in the *Check for structure and format errors* section.

In [ ]:
%%sql
WITH check_results AS (
    SELECT
        *,
        CASE
            WHEN (release_type IN ('record', 'embedded_release'))
                THEN
                    record_check
            ELSE
                release_check
        END AS results
    FROM
        release_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type <> 'compiled_release'
)

SELECT
    collection_id,
    release_type,
    additional_open_codelist_values.value -> 'codelist' AS codelist,
    codes.value AS code,
    count(*) AS occurrences
FROM
    check_results
CROSS JOIN
    jsonb_each(
        results -> 'additional_open_codelist_values'
    ) AS additional_open_codelist_values
CROSS JOIN jsonb_array_elements(value -> 'values') AS codes
GROUP BY
    collection_id,
    release_type,
    codelist,
    code
ORDER BY
    collection_id,
    release_type,
    codelist,
    count(*) DESC


### Deprecated fields

Before a field or codelist is removed from the standard, it is first marked as [deprecated](https://standard.open-contracting.org/latest/en/governance/deprecation/#deprecation).

Use this section to check for deprecated fields.

List deprecated fields:

In [ ]:
%%sql
SELECT DISTINCT ON (collection_id, path, deprecated_version, explanation)
    collection_id,
    regexp_replace(trim('"' FROM paths::text), '\/[0-9]+', '', 'g')
    || '/'
    || (deprecated_fields ->> 'field') AS path,
    deprecated_fields -> 'explanation' -> 0 AS deprecated_version,
    deprecated_fields -> 'explanation' -> 1 AS explanation,
    ocid AS example_ocid
FROM
    release_check
CROSS JOIN
    jsonb_array_elements(
        cove_output -> 'deprecated_fields'
    ) AS deprecated_fields
CROSS JOIN jsonb_array_elements(deprecated_fields -> 'paths') AS paths
INNER JOIN release ON release_check.release_id = release.id
WHERE
    collection_id IN :collection_ids;


## Calculate general statistics

Use this section to have an overview of the dataset in terms of fields published, stages covered and key summary statistics.  You can add any additional queries as needed.  

###Fields published

Calculate the fields published

In [ ]:
%%sql fields_table <<
SELECT
    path,
    distinct_releases
FROM
    field_counts
WHERE
    release_type = 'compiled_release'


In [ ]:
fields_table

Save fields table in spreadsheet:

In [ ]:
spreadsheet_name = input("Enter the name of your spreadsheet:")
save_dataframe_to_sheet(spreadsheet_name, fields_table, "fields")

### Stages covered

In [ ]:
%%sql stages <<
WITH field_counts AS (
    SELECT *
    FROM
        field_counts
    WHERE
        release_type = 'compiled_release'
        AND path IN (
            'planning',
            'tender',
            'awards',
            'contracts',
            'contracts/implementation'
        )
)

SELECT
    CASE
        WHEN paths.path = 'contracts/implementation'
            THEN
                'implementation'
        ELSE
            paths.path
    END AS stage,
    CASE
        WHEN
            paths.path IN ('planning', 'tender', 'contracts/implementation')
            THEN
                greatest(field_counts.object_property, 0)
        ELSE
            greatest(field_counts.array_count, 0)
    END AS object_count
FROM (
    SELECT
        unnest(
            ARRAY[
                'planning',
                'tender',
                'awards',
                'contracts',
                'contracts/implementation'
            ]
        ) AS path
) AS paths
LEFT JOIN field_counts USING (path)


In [ ]:
stages

In [ ]:
plot_objects_per_stage(stages)

### Number of tenders and awards by date

In [ ]:
%%sql dates <<
WITH tenders AS (
    SELECT
        extract(YEAR FROM tenderperiod_startdate) AS year,
        count(*) AS tenders
    FROM
        tender_summary
    WHERE
        release_type = 'compiled_release'
        AND tenderperiod_startdate IS NOT NULL
    GROUP BY
        year
    ORDER BY
        year
),

awards AS (
    SELECT
        extract(YEAR FROM date) AS year,
        count(*) AS awards
    FROM
        awards_summary
    WHERE
        release_type = 'compiled_release'
        AND date IS NOT NULL
    GROUP BY
        year
    ORDER BY
        year
)

SELECT
    t.year::integer,
    tenders,
    awards
FROM
    tenders AS t
FULL JOIN awards AS a ON t.year = a.year


In [ ]:
dates

In [ ]:
plot_objects_per_year(dates)

### Procurement methods used

In [ ]:
%%sql
SELECT
    procurementmethod,
    count(DISTINCT ocid),
    round((count(*) * 100.0 / sum(count(*)) OVER ()), 1) AS proportion
FROM
    tender_summary
WHERE
    release_type = 'compiled_release'
GROUP BY
    procurementmethod
ORDER BY
    proportion DESC


In [ ]:
%%sql
SELECT
    tender ->> 'procurementMethodDetails' AS method,
    count(DISTINCT ocid),
    round((count(*) * 100.0 / sum(count(*)) OVER ()), 1) AS proportion
FROM
    tender_summary
WHERE
    release_type = 'compiled_release'
GROUP BY
    method
ORDER BY
    proportion DESC


### Number of procedures by buyer

In [ ]:
%%sql buyers <<
SELECT
    identifier AS party_id,
    party -> 'name' AS name,
    count(DISTINCT ocid) AS total_tenders
FROM
    parties_summary
WHERE
    roles::text ILIKE '%%buyer%%'
    AND release_type = 'compiled_release'
GROUP BY
    identifier,
    party -> 'name'
ORDER BY
    total_tenders DESC


In [ ]:
buyers.head(10)

In [ ]:
plot_top_buyers(buyers.head(10))

## Relevance analysis

Use this section to assess if the publication contains the required fields to answer "who bought what from whom, for how much, when and how" for some subset of contracting processes.

Generate a list of the fields published:

In [ ]:
fields_list = fields_table.iloc[:, 0].tolist()

In [ ]:
relevant, result = is_relevant(fields_list)

### Does the publication pass the relevant criterion?

In [ ]:
relevant

### Why?

In [ ]:
result

### Manually check for other fields
If the main OCDS fields are not available to answer the relevant question, you should manually check if others might be used instead. This involves not only checking for the existence of a field but its content too. For example:
- If you cannot answer "who", you could check if they disclose "buyer" or "procuringEntity" roles as part of the parties array.
- If you cannot answer "from whom", you could check if they disclose the "supplier" role as part of the parties array.

If a quick check yields no alternative field, do not spend more time. If you cannot easily find the relevant field, neither will another user.

In [ ]:
fields_table

#### Save the table to a spreadsheet

In [ ]:
spreadsheet_name = input("Enter the name of your spreadsheet:")
save_dataframe_to_sheet(spreadsheet_name, result, "relevant table")